In [9]:
# Imports and configuration
%load_ext autoreload
%autoreload 2

import pandas as pd
import src.utils.file_utils as fu

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Read in raw data file
typed_hmda_data = fu.load_parquet("hmda_2024_typed")
print(typed_hmda_data.shape)

Loading dataset from /Users/c1burns/Documents/UTD/BUAN 6341/project_repo/data/interim/hmda_2024_typed.parquet
(8841112, 76)


In [12]:
# Verify target variable is binary
typed_hmda_data["denied_flag"].dtype

bool[pyarrow]

In [13]:
# Verify target variable values are all populated True/False
typed_hmda_data["denied_flag"].value_counts()

denied_flag
False    6693164
True     2147948
Name: count, dtype: int64[pyarrow]

In [14]:
# Target variable class proportions
typed_hmda_data["denied_flag"].value_counts(normalize=True)

denied_flag
False    0.75705
True     0.24295
Name: proportion, dtype: double[pyarrow]